### API TESTING

#### Docs: https://developer.riotgames.com/docs/portal#_getting-started

In [1]:
import os
from riotwatcher import LolWatcher, ApiError
from enum import Enum
import pandas as pd
from dotenv import load_dotenv
from typing import Dict, List, Tuple, Union

load_dotenv()

True

In [2]:
class ConfigEnum(Enum):
    REGION = 'eun1'
    NAME = 'Dusiciel2115'
    NO_MATCHES = 100


watcher = LolWatcher(
    api_key = os.environ.get('RIOT_API_KEY', 'not_found')
)


In [3]:
os.environ['RIOT_API_KEY']

'RGAPI-8666e3ad-d7fc-4ee3-9fda-1bf2f59f6d17'

In [4]:
def get_puuid(watcher: LolWatcher) -> str:
    """
    Returns Summoner's PUUID
    """
    general_info = watcher.summoner.by_name(
        region = ConfigEnum.REGION.value,
        summoner_name = ConfigEnum.NAME.value
    ) 
    return general_info['puuid']

PUUID = get_puuid(
    watcher=watcher
)
       

In [5]:
#can only return 100 matches max
my_matches = watcher.match.matchlist_by_puuid(
    region = ConfigEnum.REGION.value,
    count = ConfigEnum.NO_MATCHES.value,
    puuid = PUUID,
)

In [6]:
match_sample = my_matches[0], my_matches[1]

In [7]:
def get_match_data(match_id: str) -> Dict[str, Union[str, bool]]:
    
    match_data = watcher.match.by_id(
    region = ConfigEnum.REGION.value,
    match_id = match_id
    )
    return match_data['info']['participants']

    

In [11]:
match = get_match_data(my_matches[0])

In [24]:
int_features = [
    'assists', 'champExperience', 'champLevel', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets',
    'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'enemyMissingPings', 'goldEarned', 'goldSpent', 'inhibitorTakedowns',
    'inhibitorsLost', 'kills', 'largestKillingSpree', 'longestTimeSpentLiving', 'neutralMinionsKilled', 'spell1Casts', 'spell2Casts', 'spell3Casts', 'spell4Casts',
    'summonerLevel', 'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 'totalEnemyJungleMinionsKilled', 'totalHeal',
    'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead','trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken', 'turretKills', 'turretsLost',
    'visionScore', 'wardsKilled', 'wardsPlaced', 'dragonKills', 'baronKills'
]
bool_features = [
    'firstBloodKill', 'firstTowerKill', 'win'
]
features = int_features + bool_features

In [42]:
df = pd.DataFrame(
    columns = features
)

In [43]:
import copy
#works for now (extremely unoptimal)
for player in match:
    row = [player[attr] for attr in features]
    aux_df = copy.deepcopy(df)
    aux_df.loc[len(df)] = row
df = aux_df.sum()

In [45]:
df.head()

assists                      27
champExperience           14854
champLevel                   16
consumablesPurchased          6
damageDealtToBuildings     2051
dtype: int64